In [17]:
import re, json, os

base_dir = "data/FINAL/1C_0910"
data_dir = os.path.join(base_dir,'Lv2', 're')

# # lv2
# name = sorted(os.listdir(data_dir))[1]

# lv3/4
names = []
for n in sorted(os.listdir(data_dir)):
    if n.endswith('footnote'):
        names.append(n.split('_')[0])

name = names[0].split('_')[0]
origin = json.load(open(os.path.join(data_dir, name+'_footnote', name+'.json'), 'r', encoding='utf-8'))
name

'284618539'

### 뭐였지

In [ ]:
import re
import json

def extract_qna(contents):
    """
    contents: {
        'page_contents': str,
        'page': int,
        'add_info': list   # 없으면 자동 생성
    }
    반환: contents (page_contents에 플레이스홀더 치환, add_info에 QNA 추가)
    """
    raw_text = contents.get('page_contents', '')
    if not raw_text:
        return contents

    # 이미 처리된 페이지면 (플레이스홀더가 있으면) 재처리 방지
    if '{q_' in raw_text:
        return contents

    # add_info 보장
    if 'add_info' not in contents or not isinstance(contents['add_info'], list):
        contents['add_info'] = []

    # 정답 맵 (예: "01 ④")
    answers = dict(re.findall(r'(\d{2})\s*([①②③④])', raw_text))

    # 문제 블록 매칭: 번호(2자리) 시작 ~ 해설 포함, 다음 번호 또는 문서 끝까지
    block_re = re.compile(r'(\d{2}\n.*?해설.*?)(?=\n\d{2}\n|$)', re.S)
    # 옵션 라인 정규식(해설 이전에서만 사용)
    option_line_re = re.compile(r'^[①②③④][^\n]*', re.M)

    out_parts = []
    last_idx = 0
    seq = 0

    for m in block_re.finditer(raw_text):
        seq += 1
        block = m.group(1)
        start, end = m.span(1)

        # 번호
        number_match = re.match(r'\s*(\d{2})', block)
        number = number_match.group(1) if number_match else f"{seq:02d}"

        # 질문/해설 분리(첫 '해설' 기준)
        split_pos = block.find('해설')
        before_expl = block[:split_pos]
        after_expl  = block[split_pos + len('해설'):]

        # ✅ 문제 끝에 붙은 코드(예: 19-1 21-2) 추출
        caption_matches = re.findall(r'(\d+-\d+)', before_expl.splitlines()[-1])
        captions = caption_matches if caption_matches else []

        # before_expl에서 caption 코드 제거
        if captions:
            before_expl = re.sub(r'(?:\d+-\d+\s*)+$', '', before_expl.strip())   

        # 옵션 추출: 해설 이전에서만 (줄 단위)
        raw_options = option_line_re.findall(before_expl)

        # 번호 뒤 공백 1칸 강제
        options = [re.sub(r'^([①②③④])\s*', r'\1 ', opt.strip())
                   for opt in raw_options]

        # 4개 초과 시 5번째 이후는 합치기
        if len(options) > 4:
            merged = "\n".join(options[4:])
            options = options[:4] + [merged]

        # 질문 텍스트 만들기: 번호 줄/옵션 줄 제거
        # 1) 번호줄 제거
        question_part = re.sub(r'^\s*\d{2}\s*\n', '', before_expl, count=1, flags=re.M)
        # 2) 옵션 줄 제거
        question_part = option_line_re.sub('', question_part)
        # 3) 공백 줄 정리
        question_text = "\n".join([ln for ln in question_part.splitlines() if ln.strip()])

        # 해설: 끝에 "nn ①" 패턴 있으면 제거(맨 끝에서만)
        explanation = after_expl.strip()
        # ✅ 해설 끝에 "01 ④ 02 ③" 처럼 2~3개 연속 정답이 있으면 한 번에 제거
        explanation = re.sub(r'(?:\s*\d{2}\s*[①②③④]){1,3}\s*$', '', explanation)
        # ✅ 해설 안의 ①②③④ 뒤에도 공백을 강제
        explanation = re.sub(r'([①②③④])(?!\s)', r'\1 ', explanation)

        # 정답
        answer = answers.get(number, "")

        # add_info 항목 생성
        tag = f"q_{contents['page']:04}_{seq:04}"
        contents['add_info'].append({
            'tag': tag,
            'type': "question",
            'description': {
                'number': number,
                'question': question_text,
                'options': options,
                'answer': answer,
                'explanation': explanation
            },
            'caption': captions,
            'file_path': 0,
            'bbox': 0
        })

        # 출력 텍스트 재조립: 블록 대신 플레이스홀더
        out_parts.append(raw_text[last_idx:start])
        out_parts.append("{"+tag+"}")
        last_idx = end

    # 남은 꼬리 붙이기
    out_parts.append(raw_text[last_idx:])
    contents['page_contents'] = "".join(out_parts)

    return contents


In [ ]:
# new = {
#     'file_id': origin['file_id'],
#     'title': origin['title'],
#     'cat1_domain': origin['cat1_domain'],
#     'cat2_sub': origin['cat2_sub'],
#     'cat3_specific': origin['cat3_specific'],
#     'pub_date': origin['pub_date'],
#     'contents': [],
# }

for i in range(len(origin['contents'])):
    cts = origin['contents'][i]
    
    # c = extract_qna(cts)
    
    # 끝의 숫자-숫자 패턴 추출
    TAIL_PATTERN = re.compile(r"(?:\s(\d{1,2}(?:-\d+(?:⋅\d+)?)?))+\s*\W")

    # item = cts['add_info']
    for item in cts['add_info']:
        if item['type'] != 'question':
            continue
        
        if item['caption'] == []:
            print("--------------")
            q = item["description"]["question"]
            print(item['tag'], ":", item['caption'])
            m = TAIL_PATTERN.search(q)
            if not m:
                continue

            # 끝 부분 잘라서 공백 기준 분리
            tail = q[m.start():m.end()].strip()
            captions = tail.split()  # 예: "20-1⋅2 23" → ["20-1⋅2", "23"]

            item["caption"] = captions
            item["description"]["question"] = q.replace(q[m.start():m.end()], "\n")
            print(item['tag'], ":", item['caption'])

# json.dump(origin, open(os.path.join(data_dir, name+"_workbook", name+'_new.json'), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
    

## Lv2

In [18]:
import re
import copy
from typing import Dict, Any, List

PUNCT_BEFORE = r'[\.\"”\'’\)\]\}〉》」』>]'  # 번호 앞에 올 수 있는 문장부호들

def transform_list(items: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    여러 페이지를 일괄 변환.
    각 페이지당 1문항으로 가정하여 {note_<page>_0001} 형태로 변환.
    """
    return [convert_page_record(it) for it in items]

def convert_page_record(record: Dict[str, Any]) -> Dict[str, Any]:
    rec = copy.deepcopy(record)
    page = rec.get("page", "0000")
    text = rec.get("page_contents", "")

    # 1) footnote 정의 추출
    foot_def_pat = re.compile(r'^\s*___\s*(\d+)\s+(.*)$', flags=re.MULTILINE)
    foot_defs: List[tuple[str, str]] = foot_def_pat.findall(text)

    # 2) 본문에서 footnote 정의 줄 제거
    text = foot_def_pat.sub("", text)
    text = re.sub(r'\n{3,}', '\n\n', text).strip()

    add_info = rec.get("add_info", []) or []

    # 3) 본문 참조 치환 + add_info 작성
    for idx, (num, desc) in enumerate(foot_defs, start=1):
        tag = f"note_{page}_{idx:04d}"

        ref_pat = re.compile(rf'({PUNCT_BEFORE}){re.escape(num)}(?=\s)', flags=re.UNICODE)
        new_text, n_sub = ref_pat.subn(rf'\1{{{tag}}}', text, count=1)
        if n_sub == 0:
            alt_pat = re.compile(rf'(?<!\d){re.escape(num)}(?=\s)', flags=re.UNICODE)
            new_text, _ = alt_pat.subn(rf'{{{tag}}}', text, count=1)
        text = new_text

        # description에 번호를 포함해서 저장
        add_info.append({
            "tag": tag,
            "type": "footnote",
            "description": f"{num} {desc.strip()}",
            "caption": None,
            "file_path": None,
            "bbox": None
        })

    return {
        **rec,
        "page_contents": text,
        "add_info": add_info
    }


In [19]:
data_list = origin.get('contents')
converted_list = transform_list(data_list)

origin['contents'] = converted_list
json.dump(origin, open(os.path.join(data_dir, name+"_footnote", name+'_new.json'), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
